Load packages

In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Pkg
Pkg.activate(".")
using Test
using LifeInsuranceDataModel
using BitemporalPostgres
using Logging
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

setting your username and password - if not in on gitpod
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    ENV["SEARCHLIGHT_USERNAME"] = "mf"
    ENV["SEARCHLIGHT_PASSWORD"] = "mf"
    run(```psql -d postgres -f sqlsnippets/droptables.sql```)
end


Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
println(ENV["SEARCHLIGHT_USERNAME"])
println(ENV["SEARCHLIGHT_PASSWORD"])

LifeInsuranceDataModel.load_model()

In [ ]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Or just connect to an existing model

In [ ]:
LifeInsuranceDataModel.connect()

Create a Partner

In [ ]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

Create tariffs

In [ ]:
# create Tariffs
function create_tariff(dsc, mt)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc, mortality_table=mt)
    w = Workflow(type_of_entity="Tariff",
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end

LifeRiskTariff = create_tariff("Life Risk Insurance", "1980 CET - Male Nonsmoker, ANB")
TerminalIllnessTariff = create_tariff("Terminal Illness", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
OccupationalDisabilityTariff = create_tariff("Occupational Disability", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
ProfitParticipationTariff = create_tariff("Profit participation", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
LifeRiskTariff2 = create_tariff("Life Risk Insurance", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

find(TariffRevision)

In [ ]:
find(Tariff, SQLWhereExpression("id=?", ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

Create Product

In [ ]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Occupational Disablity"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

Testing

Create contract

In [ ]:
w1 = Workflow(type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
create_subcomponent!(c, cpr, cprr, w1)
# pi 1
LifeRiskTIODProduct = find(Product, SQLWhereExpression("id=?", 2))[1].id.value
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(w1, cpi, LifeRiskTIODProduct, Partner1, PartnerRole)

commit_workflow!(w1)

update Contract yellow

In [ ]:
cr1 = ContractRevision(ref_component=c.id, description="contract 1, 2nd mutation")
w2 = Workflow(type_of_entity="Contract",
    ref_history=w1.ref_history,
    tsw_validfrom=ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w2)
update_component!(cr, cr1, w2)
commit_workflow!(w2)

   update Contract red

In [ ]:

w3 = Workflow(type_of_entity="Contract",
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)


In [ ]:
update_entity!(w3)
cr1 = findcomponentrevision(ContractRevision, c.id, w3.ref_version)[1]
cr2 = ContractRevision(ref_component=c.id, description="contract 1, 3rd mutation retrospective")
update_component!(cr1, cr2, w3)
commit_workflow!(w3)


In [ ]:
w4 = Workflow(type_of_entity="Contract",
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2018, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w4)
cr3 = ContractRevision(ref_component=c.id, description="contract 1, 4th mutation")
update_component!(cr2, cr3, w4)

# pi 2
LifeRiskTIODProduct = find(Product, SQLWhereExpression("id=?", 2))[1].id.value
Partner1
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=2, ref_product=LifeRiskTIODProduct, description="from contract 4th mutation")
create_subcomponent!(c, cpi, cpir, w4)

LifeInsuranceDataModel.create_product_instance(w4, cpi, LifeRiskTIODProduct, Partner1, PartnerRole)

commit_workflow!(w4)

In [ ]:
  
LifeInsuranceDataModel.connect()

c = find(Contract)[1]
h = c.ref_history
w = Workflow(type_of_entity="Contract",
    ref_history=h,
    tsw_validfrom=ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"),
)
update_entity!(w)

committed = csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))
changed = csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))

changed.revision.description = "CR first mutation by GUI model")
changed.partner_refs[1].rev.description = "CPR first mutation by GUI model")
changed.product_items[1].revision.description = "PIR first mutation by GUI model")
changed.product_items[1].tariff_items[1].tariff_ref.rev.deferment = 9)
changed.product_items[1].tariff_items[1].tariff_ref.rev.description = "TIR first mutation by GUI model")
changed.product_items[1].tariff_items[1].partner_refs[1].rev.description = "bubu|")


In [ ]:

deltas = [
    (committed.revision, model1.revision)
    (committed.partner_refs[1].rev, model1.partner_refs[1].rev)
    (committed.product_items[1].revision, model1.product_items[1].revision)
    (committed.product_items[1].tariff_items[1].tariff_ref.rev,model1.product_items[1].tariff_items[1].tariff_ref.rev)
    (committed.product_items[1].tariff_items[1].partner_refs[1].rev,model1.product_items[1].tariff_items[1].partner_refs[1].rev)]


In [ ]:
for delta in deltas
    println(delta)
    prev = delta[1]
    curr = delta[2]
    update_component!(prev, curr, w)
end

In [ ]:
persisted = csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))

@testset "persisted pending transactions" begin
    @test(persisted.revision.description == changed.revision.description)
    @test(persisted.partner_refs[1].rev.description == changed.partner_refs[1].rev.description)
    @test(persisted.product_items[1].revision.description == changed.product_items[1].revision.description)
    @test(persisted.product_items[1].tariff_items[1].tariff_ref.rev.deferment == changed.product_items[1].tariff_items[1].tariff_ref.rev.deferment)
    @test(persisted.product_items[1].tariff_items[1].tariff_ref.rev.description == changed.product_items[1].tariff_items[1].tariff_ref.rev.description)
    @test(persisted.product_items[1].tariff_items[1].partner_refs[1].rev.description == changed.product_items[1].tariff_items[1].partner_refs[1].rev.description)
end


In [ ]:
rollback_workflow!(w)
rolledback = csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))

@testset "rolled back pending transactions" begin
    @test(rolledback.revision.description == changed.revision.description)
    @test(rolledback.partner_refs[1].rev.description == committed.partner_refs[1].rev.description)
    @test(rolledback.product_items[1].revision.description == committed.product_items[1].revision.description)
    @test(rolledback.product_items[1].tariff_items[1].tariff_ref.rev.deferment == committed.product_items[1].tariff_items[1].tariff_ref.rev.deferment)
    @test(rolledback.product_items[1].tariff_items[1].tariff_ref.rev.description == committed.product_items[1].tariff_items[1].tariff_ref.rev.description)
    @test(rolledback.product_items[1].tariff_items[1].partner_refs[1].rev.description == committed.product_items[1].tariff_items[1].partner_refs[1].rev.description)
end